In [ ]:
import sys
import os
import json
import pandas as pd

# If you installed the code from source and the import fails with the following error:
#    (ModuleNotFoundError: No module named 'solvation_predictor')
# Make sure that you have activated the correct conda environment (`conda activate env_solprop`) or 
# using the correct kernel.
# You can add the SolProp_ml path using following options:
#    Option 1: Follow the installation instruction option 2 and make sure you run `pip install -e .` line.
#    Option 2: Add the path to the `SolProp_ML` to the `PYTHONPATH` in your .bash_profile or .bashrc file
#    Option 3: Uncomment the line below. Make sure to replace `Path-To-SolProp_ML` with the actual path
#              to your SolProp_ML directory in the line below.

# sys.path.append("/Path-To-SolProp_ML/SolProp_ML") # Add SolProp_ML to the path
import solvation_predictor.calculate_solubility as calc

# Ex 1. Property prediction

Predict solvation free energy, solvation enthalpy, aqueous solid solubility, and Abraham solute parameters (E, S, A, B, L, V) at 298 K given the csv file with solute and solvent SMILES or InChI as an input

In [ ]:
df = pd.read_csv('example_property_pred/input.csv')
log_file_path = 'example_property_pred/prop_prediction_log.log'

if os.path.exists(log_file_path):
    os.remove(log_file_path)
    
predictions = calc.predict_property(
    csv_path=None, # provide either the path to the csv file or the pandas dataframe object
    df=df, # provide either the path to the csv file or the pandas dataframe object
    gsolv=True, # predict solvation free energy at 298 K
    hsolv=True, # predict solvation enthalpy at 298 K
    saq=True, # predict aqueous solid solubility at 298 K
    solute_parameters=True, # predict Abraham solid parameters at 298 K
    reduced_number=False, # use a reduced number of models for faster but less accurate prediction (does not work for solute_parameters)
    validate_data_list=['solute', 'solvent'], # a list of data names to validate (also converts inchis to smiles)
    export_csv='example_property_pred/results.csv', # path if csv file with predictions needs to be exported
    logger=log_file_path # path to the logger file if the logger file needs to be exported
)


# Ex 2. Solid solubility prediction with reference solvent, reference solubility, and temperature

In [ ]:
df = pd.read_csv('example_solubility_pred_all_options/input.csv')
log_file_path = 'example_solubility_pred_all_options/solubility_prediction_log.log'

if os.path.exists(log_file_path):
    os.remove(log_file_path)
    
results = calc.calculate_solubility(
    path=None, # provide either the path to the csv file or the pandas dataframe object
    df=df, # provide either the path to the csv file or the pandas dataframe object
    validate_data_list=['solute', 'solvent', 'reference_solvent', 'reference_solubility', 'temperature'], # a list of data names to validate (also converts inchis to smiles)
    calculate_aqueous=True, # also calculate aqueous solubility even if reference solubility is provided
    calculate_Hdiss_T_dep=True, # calculate solubility with the temperature dependent dissolution enthalpy (method 2)
    reduced_number=False, # use a reduced number of models for faster but less accurate prediction (does not work for solute_parameters)
    export_csv='example_solubility_pred_all_options/results.csv', # path if csv file with predictions needs to be exported
    export_detailed_csv=True, # True if csv file with all predictions and calculations needs to be exported
    logger=log_file_path # path to the logger file if the logger file needs to be exported
)


# Ex 3. Solid solubility prediction only at 298 K with reference solvent and reference solubility. No temperature-dependent solubility calculation.

In [ ]:
df = pd.read_csv('example_only_298K/input.csv')
log_file_path = 'example_only_298K/solubility_prediction_log.log'

if os.path.exists(log_file_path):
    os.remove(log_file_path)
    
results = calc.calculate_solubility(
    path=None, # provide either the path to the csv file or the pandas dataframe object
    df=df, # provide either the path to the csv file or the pandas dataframe object
    validate_data_list=['solute', 'solvent', 'reference_solvent', 'reference_solubility'], # a list of data names to validate (also converts inchis to smiles)
    calculate_aqueous=True, # also calculate aqueous solubility even if reference solubility is provided
    calculate_Hdiss_T_dep=False, # calculate solubility with the temperature dependent dissolution enthalpy (method 2)
    reduced_number=False, # use a reduced number of models for faster but less accurate prediction (does not work for solute_parameters)
    export_csv='example_only_298K/results.csv', # path if csv file with predictions needs to be exported
    export_detailed_csv=True, # True if csv file with all predictions and calculations needs to be exported
    logger=log_file_path # path to the logger file if the logger file needs to be exported
)


# Ex 4. Solid solubility prediction only at various temperatures without reference solvent.

In [ ]:
df = pd.read_csv('example_no_ref_solvent/input.csv')
log_file_path = 'example_no_ref_solvent/solubility_prediction_log.log'

if os.path.exists(log_file_path):
    os.remove(log_file_path)
    
results = calc.calculate_solubility(
    path=None, # provide either the path to the csv file or the pandas dataframe object
    df=df, # provide either the path to the csv file or the pandas dataframe object
    validate_data_list=['solute', 'solvent', 'temperature'], # a list of data names to validate (also converts inchis to smiles)
    calculate_aqueous=True, # also calculate aqueous solubility even if reference solubility is provided
    calculate_Hdiss_T_dep=True, # calculate solubility with the temperature dependent dissolution enthalpy (method 2)
    reduced_number=False, # use a reduced number of models for faster but less accurate prediction (does not work for solute_parameters)
    export_csv='example_no_ref_solvent/results.csv', # path if csv file with predictions needs to be exported
    export_detailed_csv=True, # True if csv file with all predictions and calculations needs to be exported
    logger=log_file_path # path to the logger file if the logger file needs to be exported
)


# Ex 5. Solid solubility prediction with reference solvent, reference solubility, and temperature and with an input solvent critical property dictionary

`logS_T_with_T_dep_Hdiss` output (when using `calculate_Hdiss_T_dep` option) is only availabl for only around 100 common solvents whose critical temperture and critical density are known. 
A user can provide his/her own dictionary (`solv_crit_prop_dict`) containing the critical temperature and critical density of a solvent to use get the `logS_T_with_T_dep_Hdiss` output for other solvents.


<b>`solv_crit_prop_dict`</b>: (optional input) dictionary containing the CoolProp name, critical temperature (in K), and critical density (in mol/m3) of solvents. If this is not provided, default dictionary is used. The inchi with fixed H option is used as a dictionary key.

- example format of solv_crit_prop_dict:

   {'InChI=1/CH4O/c1-2/h2H,1H3': {'name': 'methanol', 'smiles': 'CO', 'coolprop_name': 'Methanol', 'Tc': 513.0, 'rho_c': 8510.0},
   'InChI=1/C3H8O/c1-3(2)4/h3-4H,1-2H3': {'name': 'propan-2-ol',  'smiles': 'CC(C)O', 'coolprop_name': None, 'Tc': 509.0, 'rho_c': 4500.0}}

In [ ]:
crit_data_path = 'example_with_manual_solv_crit_dict/solvent_crit_data_more.json'
with open(crit_data_path) as f:
    solv_crit_prop_dict = json.load(f)  # inchi with fixed H is used as a solvent key
    
df = pd.read_csv('example_with_manual_solv_crit_dict/input.csv')
log_file_path = 'example_with_manual_solv_crit_dict/solubility_prediction_log.log'

if os.path.exists(log_file_path):
    os.remove(log_file_path)

results = calc.calculate_solubility(
    path=None, # provide either the path to the csv file or the pandas dataframe object
    df=df, # provide either the path to the csv file or the pandas dataframe object
    validate_data_list=['solute', 'solvent', 'reference_solvent', 'reference_solubility', 'temperature'], # a list of data names to validate (also converts inchis to smiles)
    calculate_aqueous=True, # also calculate aqueous solubility even if reference solubility is provided
    calculate_Hdiss_T_dep=True, # calculate solubility with the temperature dependent dissolution enthalpy (method 2)
    reduced_number=False, # use a reduced number of models for faster but less accurate prediction (does not work for solute_parameters)
    export_csv='example_with_manual_solv_crit_dict/results.csv', # path if csv file with predictions needs to be exported
    export_detailed_csv=True, # True if csv file with all predictions and calculations needs to be exported
    solv_crit_prop_dict = solv_crit_prop_dict,  # dictionary containing the CoolProp name, critical temperature (in K),
                                                # and critical density (in mol/m3) of solvents.
    logger=log_file_path # path to the logger file if the logger file needs to be exported
)
